### Problem Statement:

##### To understand which clients are more likely to buy the short term deposit so as to target the audience effectively and effeciently.    

In [1]:
# pip install xgboost==1.2.0

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from plotly.offline import plot
import plotly
import plotly.offline as pyoff
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
import os


from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split #split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import accuracy_score #metrics
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

#tools for hyperparameters search
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


In [13]:
os.getcwd()

'C:\\Users\\User\\Desktop\\Insofe'

In [14]:
# READING THE DATASET via pickle
#df_sales = pd.read_pickle('sales_df.pkl')
df = pd.read_csv('Insofe.csv',sep=',')
test_data = pd.read_csv('test_cases.csv',sep=',')
print(df.head())
print(test_data.head())
df.columns

   age        job  marital    education credit_default housing loan  \
0   56  housemaid  married        basic             no      no   no   
1   57   services  married  high.school        unknown      no   no   
2   37   services  married  high.school             no     yes   no   
3   40     admin.  married        basic             no      no   no   
4   56   services  married  high.school             no      no  yes   

     contact contacted_month day_of_week  ...  previous     poutcome  \
0  telephone             may         mon  ...         0  nonexistent   
1  telephone             may         mon  ...         0  nonexistent   
2  telephone             may         mon  ...         0  nonexistent   
3  telephone             may         mon  ...         0  nonexistent   
4  telephone             may         mon  ...         0  nonexistent   

   emp_var_rate  cons_price_idx cons_conf_idx  euribor3m  nr_employed   y  \
0           1.1          93.994         -36.4      4.857       

Index(['age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan', 'contact', 'contacted_month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y',
       'Binned_by_mins', 'AgeGroup'],
      dtype='object')

In [15]:
df.dtypes

age                  int64
job                 object
marital             object
education           object
credit_default      object
housing             object
loan                object
contact             object
contacted_month     object
day_of_week         object
duration             int64
campaign             int64
pdays                int64
previous             int64
poutcome            object
emp_var_rate       float64
cons_price_idx     float64
cons_conf_idx      float64
euribor3m          float64
nr_employed        float64
y                   object
Binned_by_mins      object
AgeGroup            object
dtype: object

## Split into categorical and numerical attributes

In [16]:
cat_attr = list(df.select_dtypes("category").columns)
num_attr = list(df.columns.difference(cat_attr))

cat_attr.pop(-2)#popping out the target variable

IndexError: pop from empty list

## Instantiate Pre-processing Objects for Pipeline

In [ ]:

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent',fill_value="missing_value")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

## Train-Test Split

In [19]:
X_train, y_train = df.loc[:,df.columns!='y'], df.loc[:,'y']


X_test, y_test = test_data.loc[:,test_data.columns!='y'], test_data.loc[:,'y']
X_train.head()

,age,job,marital,education,credit_default,housing,loan,contact,contacted_month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,Binned_by_mins,AgeGroup
0,56,housemaid,married,basic,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Less than 2min,Married
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Less than 2min,Married
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Less than 2min,Married
3,40,admin.,married,basic,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Less than 2min,Married
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,Less than 2min,Married


## Instantiate Pipeline Object

In [ ]:
clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

## Build Logistic Regression Model

In [ ]:
clf_logreg.fit(X_train, y_train)

## Evaluate Model

In [ ]:
train_pred = clf_logreg.predict(X_train)
test_pred = clf_logreg.predict(X_test)

print(classification_report(y_test, test_pred))

## K-Nearest Neighbour

In [ ]:
%%time
knn = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2))])

knn.fit(X_train,y_train)

In [ ]:
train_pred =knn.predict(X_train)
test_pred = knn.predict(X_test)

print(classification_report(y_test, test_pred))

## Naive Bayes

In [ ]:
%%time
gnb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',  GaussianNB())])


gnb.fit(X_train,y_train)

In [ ]:
train_pred = gnb.predict(X_train)
test_pred = gnb.predict(X_test)

print(classification_report(y_test, test_pred))

## Build SVC Model
### A. SVM (Linear  and RBF Models)

In [ ]:

%%time
svc_lin = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC())])
svc_lin.set_params(classifier__kernel='linear',classifier__C=1,
                   classifier__random_state=123)
svc_lin.fit(X_train,y_train)

In [ ]:
train_pred =svc_lin.predict(X_train)
test_pred = svc_lin.predict(X_test)

print(classification_report(y_test, test_pred))

### B. SVM (rbf kernel)

In [ ]:
%%time
svc_rbf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC())])
svc_rbf.set_params(classifier__kernel='rbf',classifier__C=1,
                   classifier__gamma=0.01,classifier__random_state=123)

svc_rbf.fit(X_train,y_train)

In [ ]:
train_pred =svc_rbf.predict(X_train)
test_pred = svc_rbf.predict(X_test)

print(classification_report(y_test, test_pred))

## SVM (Grid Search CV)

In [ ]:
clf_svc = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC())])
svc_param_grid = {'classifier__C': [0.001], 
                  'classifier__gamma':[0.1], 
                 "classifier__kernel":['rbf']}
svc_grid = GridSearchCV(clf_svc, param_grid=svc_param_grid, cv=5)

In [ ]:
svc_grid

In [ ]:
%%time
svc_grid.fit(X_train,y_train)

In [ ]:
svc_grid.best_params_

In [ ]:
train_pred = svc_grid.predict(X_train)
test_pred = svc_grid.predict(X_test)

print(classification_report(y_test, test_pred))

## Decision Tree

In [ ]:
%%time
clf_dt = Pipeline(steps=[('classifier', DecisionTreeClassifier())])

dt_param_grid = {'classifier__criterion': ['entropy', 'gini'], 'classifier__max_depth': [6,8,10,12], 
                 "classifier__min_samples_split": [2, 10, 20],"classifier__min_samples_leaf": [1, 5, 10]}

dt_grid_bal = GridSearchCV(clf_dt, param_grid=dt_param_grid, cv=5)

dt_grid_bal.fit(X_train,y_train)

In [ ]:
dt_grid.best_params_

In [ ]:
train_pred = svc_grid.predict(X_train)
test_pred = svc_grid.predict(X_test)

print(classification_report(y_test, test_pred))

## XGBoost Model 

In [ ]:
#train XGBoost model
xgb = xgb.XGBClassifier(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train.squeeze().values)

#calculate and print scores for the model for top 15 features
y_train_preds = xgb.predict(X_train)
y_test_preds = xgb.predict(X_test)

print('XGB accuracy score for train: %.3f: test: %.3f' % (
        accuracy_score(y_train, y_train_preds),
        accuracy_score(y_test, y_test_preds)))
cm = confusion_matrix(y_test, y_test_preds)
# cm
# # calculate prediction
# precision = precision_score(y_true, y_pred, average='binary')
# print('Precision: %.3f' % precision)

print(classification_report(y_test, y_test_preds))


In [ ]:
#get feature importances from the model
headers = ["name", "score"]
values = sorted(zip(X_train.columns, xgb.feature_importances_), key=lambda x: x[1] * -1)
xgb_feature_importances = pd.DataFrame(values, columns = headers)

#plot feature importances
x_pos = np.arange(0, len(xgb_feature_importances))
plt.bar(x_pos, xgb_feature_importances['score'])
plt.xticks(x_pos, xgb_feature_importances['name'])
plt.xticks(rotation=90)
plt.title('Feature importances (XGB)')

plt.show()

## Build Random Forest

In [21]:
Rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',RandomForestClassifier(random_state=42))])

NameError: name 'preprocessor' is not defined

In [ ]:
Rf.fit(X_train, y_train)
print(classification_report(y_test, y_test_preds))

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
)
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

In [ ]:
result = permutation_importance(
    model, X_train, y_train, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_train.columns[sorted_idx]
)
ax.set_title("Permutation Importances (train set)")
fig.tight_layout()
plt.show()